# Pipeline de Detecção de Reclamações por Falta de Contato

In [60]:
import pandas as pd
import unicodedata
import re
import itertools

In [61]:
# Função de pré-processamento
def preprocess_text(text):
    text = text.lower()
    text = unicodedata.normalize('NFKD', text)
    text = ''.join(c for c in text if not unicodedata.combining(c))
    text = re.sub(r'[^a-z\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [62]:
# Exemplo de uso com dados fictícios
data = {'Texto_BC': [
    'Não tive retorno nenhum da agência sobre meu pedido.',
    'Enviei um email e não recebi feedback do gerente.',
    'Gostaria de saber sobre o status.',
    'Ainda não recebi nenhum feedback da agência referente ao contrato.',
    'Estou sem retorno do gerente após enviar a documentação.',
    'A agência não entrou em contato comigo desde a última reunião.',
    'Preciso de um contato do gerente para avançar com o processo.',
    'Enviei várias mensagens, mas a agência não me deu nenhum retorno.',
    'Sem retorno do gerente, não posso prosseguir com o serviço.',
    'Minha dúvida é sobre o cronograma e não tive retorno da agência.'
]}

df = pd.DataFrame(data)
df['text_tratado'] = df['Texto_BC'].apply(preprocess_text)
df

,Texto_BC,text_tratado
0,Não tive retorno nenhum da agência sobre meu p...,nao tive retorno nenhum da agencia sobre meu p...
1,Enviei um email e não recebi feedback do gerente.,enviei um email e nao recebi feedback do gerente
2,Gostaria de saber sobre o status.,gostaria de saber sobre o status
3,Ainda não recebi nenhum feedback da agência re...,ainda nao recebi nenhum feedback da agencia re...
4,Estou sem retorno do gerente após enviar a doc...,estou sem retorno do gerente apos enviar a doc...
5,A agência não entrou em contato comigo desde a...,a agencia nao entrou em contato comigo desde a...
6,Preciso de um contato do gerente para avançar ...,preciso de um contato do gerente para avancar ...
7,"Enviei várias mensagens, mas a agência não me ...",enviei varias mensagens mas a agencia nao me d...
8,"Sem retorno do gerente, não posso prosseguir c...",sem retorno do gerente nao posso prosseguir co...
9,Minha dúvida é sobre o cronograma e não tive r...,minha duvida e sobre o cronograma e nao tive r...


In [63]:
# Possíveis frases que indicam falta de retorno
frases = [
    'nao tive retorno',
    'nenhum contato',
    'falta de feedback'
]
sinonimos = [
    ['contato', 'retorno', 'feedback', 'resposta'],
    ['nenhum', 'sem', 'falta de', 'nao tive'],
    ['recebi', 'obtive', 'tive'],
    ['atendimento', 'suporte', 'apoio'],
    ['ausencia', 'falta', 'sem'],
]

# Frases que indicam espera ou demora
frases_espera = [
    'ha dias',
    'ha semanas',
    'ha meses',
    'desde o dia',
    'ate agora nada',
    'estou esperando'
]
sinonimos_espera = [
    ['ha', 'faz', 'fazem'],
    ['dia', 'dias', 'semana', 'semanas', 'mês', 'meses'],
    ['desde', 'desde o dia', 'a partir de'],
    ['ate agora nada', 'ate o momento nada', 'ate agora nenhuma resposta'],
    ['estou esperando', 'aguardando', 'fico aguardando']
]

# Frases que indicam certeza sobre a falta de retorno
frases_certeza = [
    'agencia nao me retornou',
    'agencia nao me deu resposta',
    'ate agora a agencia nao me deu respostas',
    'o gerente nao me deu retorno',
    'funcionario da agencia nao respondeu',
    'o gerente nao me respondeu',
    'ate o momento o funcionario da agencia nao me deu retorno'
]
sinonimos_certeza = [
    ['gerente', 'funcionario da agencia', 'agencia'],
    ['resposta', 'retorno', 'parecer']
]

# pre processamento
frases = [preprocess_text(fr) for fr in frases]
sinonimos = [[preprocess_text(term) for term in group] for group in sinonimos]
frases_espera = [preprocess_text(fr) for fr in frases_espera]
sinonimos_espera = [[preprocess_text(term) for term in group] for group in sinonimos_espera]
frases_certeza = [preprocess_text(fr) for fr in frases_certeza]
sinonimos_certeza = [[preprocess_text(term) for term in group] for group in sinonimos_certeza]

In [64]:
# Monta dicionários de lookup rápido
sinonimos_map = {palavra: grupo for grupo in sinonimos for palavra in grupo}
sinonimos_espera_map = {term: group for group in sinonimos_espera for term in group}
sinonimos_certeza_map = {palavra: grupo for grupo in sinonimos_certeza for palavra in grupo}

# Expansão de frases gerais
frases_expandidas = []
for frase in frases:
    tokens = frase.split()
    listas = [sinonimos_map.get(tok, [tok]) for tok in tokens]
    for combo in itertools.product(*listas):
        frases_expandidas.append(' '.join(combo))
frases_expandidas = list(set(frases_expandidas))

# Expansão das frases de espera
expandidas_espera = []
for frase in frases_espera:
    tokens = frase.split()
    listas = [sinonimos_espera_map.get(tok, [tok]) for tok in tokens]
    for combo in itertools.product(*listas):
        expandidas_espera.append(' '.join(combo))
expandidas_espera = list(set(expandidas_espera))

# Expansão de frases de certeza
expandidas_certeza = []
for frase in frases_certeza:
    tokens = frase.split()
    listas = [sinonimos_certeza_map.get(tok, [tok]) for tok in tokens]
    for combo in itertools.product(*listas):
        expandidas_certeza.append(' '.join(combo))
expandidas_certeza = list(set(expandidas_certeza))

print("Frases expandidas (possíveis falta de retorno):")
print(frases_expandidas)
print("\nFrases expandidas de possíveis espera:")
print(expandidas_espera)
print("\nFrases expandidas de certeza:")
print(expandidas_certeza)

Frases expandidas (possíveis falta de retorno):
['nao obtive contato', 'falta de retorno', 'ausencia de retorno', 'nao recebi retorno', 'nao recebi contato', 'nenhum feedback', 'nenhum contato', 'sem retorno', 'nenhum retorno', 'ausencia de resposta', 'falta de resposta', 'nao tive retorno', 'ausencia de feedback', 'nao obtive retorno', 'sem de contato', 'falta de contato', 'sem de retorno', 'nao obtive feedback', 'sem resposta', 'nao recebi feedback', 'sem de feedback', 'nao obtive resposta', 'sem contato', 'sem feedback', 'nenhum resposta', 'nao tive contato', 'ausencia de contato', 'nao tive feedback', 'nao tive resposta', 'sem de resposta', 'falta de feedback', 'nao recebi resposta']

Frases expandidas de possíveis espera:
['desde o dias', 'desde o dia o dia', 'ha dias', 'a partir de o semana', 'estou esperando', 'faz mes', 'desde o semanas', 'fazem semana', 'faz meses', 'desde o dia', 'a partir de o meses', 'fazem dia', 'fazem mes', 'desde o dia o semanas', 'a partir de o dias', '

In [65]:
def score_text(text):
    text = preprocess_text(text)
    tokens = text.split()

    # 1) Se não mencionar 'agencia', score = 0
    if 'agencia' not in tokens:
        return 0.0

    # 2) Se mencionar alguma frase de certeza, retorna 1.0 imediatamente
    for frase in expandidas_certeza:
        f_tokens = frase.split()
        for i in range(len(tokens) - len(f_tokens) + 1):
            if tokens[i:i+len(f_tokens)] == f_tokens:
                return 1.0

    # 3) Score base por menções
    score = 0.25  # já sabemos que 'agencia' está presente
    if 'gerente' in tokens:
        score += 0.25
    
    # 3.5) Se mencionar alguma frase de espera, soma 1.5 ao score
    for frase in expandidas_espera:
        f_tokens = frase.split()
        for i in range(len(tokens) - len(f_tokens) + 1):
            if tokens[i:i+len(f_tokens)] == f_tokens:
                score += 1.5
                break

    # 4) Matching das frases gerais com bônus por proximidade
    for frase in frases_expandidas:
        f_tokens = frase.split()
        for i in range(len(tokens) - len(f_tokens) + 1):
            if tokens[i:i+len(f_tokens)] == f_tokens:
                # calcula distância mínima até 'agencia' ou 'gerente'
                positions = range(i, i + len(f_tokens))
                min_dist = None
                for pos in positions:
                    for kw in ['agencia', 'gerente']:
                        kw_positions = [j for j, t in enumerate(tokens) if t == kw]
                        for kp in kw_positions:
                            dist = abs(pos - kp)
                            if min_dist is None or dist < min_dist:
                                min_dist = dist
                # aplica bônus
                if min_dist is not None:
                    if min_dist <= 2:
                        bonus = 0.5
                    else:
                        bonus = max(0, 0.5 * (1 - (min_dist - 2) / 8))
                    score += bonus

    # 5) Limita o score máximo a 1.0
    return min(score, 1.0)

In [66]:
# Exemplo de scoring final
df['score'] = df['Texto_BC'].apply(score_text)
df

,Texto_BC,text_tratado,score
0,Não tive retorno nenhum da agência sobre meu p...,nao tive retorno nenhum da agencia sobre meu p...,0.6875
1,Enviei um email e não recebi feedback do gerente.,enviei um email e nao recebi feedback do gerente,0.0000
2,Gostaria de saber sobre o status.,gostaria de saber sobre o status,0.0000
3,Ainda não recebi nenhum feedback da agência re...,ainda nao recebi nenhum feedback da agencia re...,0.7500
4,Estou sem retorno do gerente após enviar a doc...,estou sem retorno do gerente apos enviar a doc...,0.0000
5,A agência não entrou em contato comigo desde a...,a agencia nao entrou em contato comigo desde a...,0.2500
6,Preciso de um contato do gerente para avançar ...,preciso de um contato do gerente para avancar ...,0.0000
7,"Enviei várias mensagens, mas a agência não me ...",enviei varias mensagens mas a agencia nao me d...,0.6250
8,"Sem retorno do gerente, não posso prosseguir c...",sem retorno do gerente nao posso prosseguir co...,0.0000
9,Minha dúvida é sobre o cronograma e não tive r...,minha duvida e sobre o cronograma e nao tive r...,0.7500


In [67]:
# função para criar novas features
def calcular_score_proximidade(tokens, lista_frases):
    """Retorna o bônus de proximidade máximo (0 a 0.5) entre tokens e frases."""
    bonus_maximo = 0.0
    for frase in lista_frases:
        tokens_frase = frase.split()
        tamanho = len(tokens_frase)
        for i in range(len(tokens) - tamanho + 1):
            if tokens[i:i+tamanho] == tokens_frase:
                # calcula distância mínima até 'agencia' ou 'gerente'
                dist_minima = None
                for pos in range(i, i + tamanho):
                    for chave in ['agencia', 'gerente']:
                        posicoes_chave = [j for j, t in enumerate(tokens) if t == chave]
                        for pc in posicoes_chave:
                            dist = abs(pos - pc)
                            if dist_minima is None or dist < dist_minima:
                                dist_minima = dist
                # aplica bônus proporcional
                if dist_minima is not None:
                    if dist_minima <= 2:
                        bonus = 0.5
                    else:
                        bonus = max(0, 0.5 * (1 - (dist_minima - 2) / 8))
                    bonus_maximo = max(bonus_maximo, bonus)
    return bonus_maximo

def contem_frase(tokens, lista_frases):
    """Retorna 1 se qualquer frase da lista for encontrada nos tokens, senão 0."""
    for frase in lista_frases:
        tokens_frase = frase.split()
        tamanho = len(tokens_frase)
        for i in range(len(tokens) - tamanho + 1):
            if tokens[i:i+tamanho] == tokens_frase:
                return 1
    return 0


# 1) Cria coluna temporária de tokens
df['tokens'] = df['text_tratado'].apply(lambda texto: texto.split())

# 2) Novas features
df['contem_agencia']        = df['tokens'].apply(lambda tokens: int('agencia' in tokens))
df['contem_gerente']        = df['tokens'].apply(lambda tokens: int('gerente' in tokens))
df['score_proximidade']     = df['tokens'].apply(lambda tokens: calcular_score_proximidade(tokens, frases_expandidas))
df['contem_frase_espera']   = df['tokens'].apply(lambda tokens: contem_frase(tokens, expandidas_espera))
df['contem_frases_certeza'] = df['tokens'].apply(lambda tokens: contem_frase(tokens, expandidas_certeza))

# 3) Remove coluna auxiliar
df = df.drop(columns=['tokens'])
df

,Texto_BC,text_tratado,score,contem_agencia,contem_gerente,score_proximidade,contem_frase_espera,contem_frases_certeza
0,Não tive retorno nenhum da agência sobre meu p...,nao tive retorno nenhum da agencia sobre meu p...,0.6875,1,0,0.4375,0,0
1,Enviei um email e não recebi feedback do gerente.,enviei um email e nao recebi feedback do gerente,0.0000,0,1,0.5000,0,0
2,Gostaria de saber sobre o status.,gostaria de saber sobre o status,0.0000,0,0,0.0000,0,0
3,Ainda não recebi nenhum feedback da agência re...,ainda nao recebi nenhum feedback da agencia re...,0.7500,1,0,0.5000,0,0
4,Estou sem retorno do gerente após enviar a doc...,estou sem retorno do gerente apos enviar a doc...,0.0000,0,1,0.5000,0,0
5,A agência não entrou em contato comigo desde a...,a agencia nao entrou em contato comigo desde a...,0.2500,1,0,0.0000,0,0
6,Preciso de um contato do gerente para avançar ...,preciso de um contato do gerente para avancar ...,0.0000,0,1,0.0000,0,0
7,"Enviei várias mensagens, mas a agência não me ...",enviei varias mensagens mas a agencia nao me d...,0.6250,1,0,0.3750,0,0
8,"Sem retorno do gerente, não posso prosseguir c...",sem retorno do gerente nao posso prosseguir co...,0.0000,0,1,0.5000,0,0
9,Minha dúvida é sobre o cronograma e não tive r...,minha duvida e sobre o cronograma e nao tive r...,0.7500,1,0,0.5000,0,0


In [1]:
from gensim.models import Word2Vec

# 1) Defina um pequeno corpus de frases já pré‑processadas (lowercase, sem acentos, só palavras)
corpus = [
    'nao tive retorno nenhum da agencia',
    'a agencia ainda nao me retornou nada',
    'falta de feedback do gerente da agencia',
    'enviei email ha dias e sem resposta',
    'estou esperando retorno desde o dia 01',
    'o gerente nao me deu nenhum parecer'
]

# 2) Converta cada frase em lista de tokens
sentences = [frase.split() for frase in corpus]

# 3) Treine o modelo (skip‑gram)
modelo = Word2Vec(
    sentences,
    vector_size=50,   # dimensão dos embeddings
    window=3,         # contexto +/- 3 palavras
    min_count=1,      # incluir até termos raros
    sg=1,             # 1 = skip‑gram, 0 = CBOW
    epochs=20
)

# 4) Vetor de uma palavra
vetor_agencia = modelo.wv['agencia']
print("Vetor (dimensão 5/50) de 'agencia':", vetor_agencia[:5], "...")

# 5) Palavras mais similares a 'contato' (aqui, ele vai usar 'contato' se existir; senão outro termo)
#    como no toy‑corpus não temos 'contato', vamos ver para 'retorno'
print("Top‑3 similares a 'retorno':", modelo.wv.most_similar('retorno', topn=3))

# 6) Similaridade entre termos
sim = modelo.wv.similarity('agencia', 'gerente')
print(f"Similaridade agencia ↔ gerente: {sim:.2f}")


Vetor (dimensão 5/50) de 'agencia': [-0.01636052  0.00906307 -0.00825356  0.0016383   0.01704748] ...
Top‑3 similares a 'retorno': [('enviei', 0.2282908707857132), ('sem', 0.20391406118869781), ('dia', 0.19164271652698517)]
Similaridade agencia ↔ gerente: 0.08
